# Proyecto TA

In [11]:
import pandas as pd 
import numpy as np
import ta 
from ta.trend import MACD
from ta.momentum import RSIIndicator 
from typing import List

In [23]:
# Data
train_d = pd.read_csv('Files/aapl_5m_train.csv')
validation_d = pd.read_csv('Files/aapl_5m_validation.csv')

In [25]:
#Signal generation 
def gen_signals(strat:List[int],signal_type:str,*args):
    ct = 0
    strat_args = args
    macd,sma,rsi = strat
    df = pd.DataFrame()
    
    if sma:
        long = strat_args[ct]
        short = strat_args[ct+1]
        ct += 2
        
        short_sma = ta.trend.SMAIndicator(train_d.Close,window=long) 
        long_sma = ta.trend.SMAIndicator(train_d.Close,window=short)
        
        train_d["short_sma"] = short_sma.sma_indicator()
        train_d["long_sma"] = long_sma.sma_indicator()    
    
    if macd:
        slow_period = 26 
        fast_period = 12
        signal_period = 9
        
        macd = MACD(train_d['Close'], window_slow=slow_period, window_fast=fast_period, window_sign=signal_period)
        train_d['MACD'] = macd.macd()
        train_d['Signal'] = macd.macd_signal()
        
    if rsi:
        rsi_period = strat_args[ct]
        ct += 1
        
        rsi_indicator = RSIIndicator(train_d['Close'],window=rsi_period)
        train_d['RSI'] = rsi_indicator.rsi()

        
    #Generar los BUY/SELL dependiendo del indicador
    if signal_type == "BUY":
        
        train_d['SMA_Buy_Signal'] = 0
        train_d.loc[train_d['long_sma'] > train_d['short_sma'], 'SMA_Buy_Signal'] = 1

        train_d['MACD_Buy_Signal'] = 0
        train_d.loc[(train_d['MACD'] > train_d['Signal']) & (train_d['MACD'] > 0), 'MACD_Buy_Signal'] = 1

        train_d['RSI_Buy_Signal'] = 0
        train_d.loc[train_d['RSI'] > 70, 'RSI_Buy_Signal'] = 1

    elif signal_type == "SELL":
        
        train_d['SMA_Sell_Signal'] = 0
        train_d.loc[train_d['long_sma'] < train_d['short_sma'], 'SMA_Sell_Signal'] = 1

        train_d['MACD_Sell_Signal'] = 0
        train_d.loc[(train_d['MACD'] < train_d['Signal']) & (train_d['MACD'] < 0), 'MACD_Sell_Signal'] = 1

        train_d['RSI_Sell_Signal'] = 0
        train_d.loc[train_d['RSI'] < 30, 'RSI_Sell_Signal'] = 1

    return train_d   

In [ ]:
#Ejemplo
strategy = [1,1,1]  # 1 MACD, 1 SMA, 1 RSI
signal_type = "BUY"  
sma_args = [50, 20]  # Long and Short SMA periods
rsi_args = [14]  # RSI period

result_df = gen_signals(strategy, signal_type, *sma_args, *rsi_args) 

In [3]:
# Combinations (Strategies)
n = list(range(1, 2**5))
combinations = list(map(lambda x: [int(bit) for bit in f"{x:05b}"], n))

In [5]:
#Strategy Backtesting
comission = .0025
stop_loss = .035
take_profit = .035
cash = 1000000


# Checar a partir esto
def backtest(strat,sl,tp,*strat_args):
    positions = []
    operations = []
    
#     strategy_signals = gen_signals(strat,signal_type:str,*strat_args)
    
#     for time, row in strategy_signals,iterations()
#         if row.sum() = len(row)

In [6]:
# Parameter Optimization
def minimize_port(x: np.array, byte_array: List[int]):
    sl, tp, *strat_args = x
    v = backtest(byte_array, sl, tp, strat_args)
    return -v

In [7]:
#1 Select strategies
#2 Trade Signals(Buy/sell)
#3 Backtesting-> Portafolio Val
#4 Max Profit (SL 1-5%, TP 1-5%, PL1 1-50, PL2 51,200) #Stop loss / take profit 
#5 Params 
#6 Test all possible combinations
#7 Select optimal strategy
#8 Backtest with data